https://www.kaggle.com/c/bioresponse

In [1]:
from utils import css_from_file
css_from_file('style/style.css')

In [2]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/home/batman/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Read data from the file __data/boehringer/(train|test).csv__ from the data folder. 

The first column is a binary variable that you want to predict. The rest are numericals

In [3]:
def load(path):
    df = pd.read_csv(path)
    if "Activity" not in df.columns:
        df["Activity"] = np.nan
    return df.drop("Activity",axis=1), df.Activity
    
X_tr, y_tr = load("data/boehringer/train.csv")
X_te, y_te = load("data/boehringer/test.csv")

print("training data shape", X_tr.shape)
print("testing data shape", X_te.shape)

training data shape (3751, 1776)
testing data shape (2501, 1776)


In [40]:
y_te

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
        ..
2471   NaN
2472   NaN
2473   NaN
2474   NaN
2475   NaN
2476   NaN
2477   NaN
2478   NaN
2479   NaN
2480   NaN
2481   NaN
2482   NaN
2483   NaN
2484   NaN
2485   NaN
2486   NaN
2487   NaN
2488   NaN
2489   NaN
2490   NaN
2491   NaN
2492   NaN
2493   NaN
2494   NaN
2495   NaN
2496   NaN
2497   NaN
2498   NaN
2499   NaN
2500   NaN
Name: Activity, Length: 2501, dtype: float64

In [4]:
X_tr.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,0.243144,...,0,0,0,0,0,0,0,0,0,0
1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,0.106480,...,1,1,1,1,0,1,0,0,1,0
2,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,0.352308,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,0.208989,...,0,0,0,0,0,0,0,0,0,0
4,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,0.125177,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_tr.describe()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,...,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000
mean,0.076948,0.592436,0.068142,0.038990,0.212112,0.686653,0.274713,0.455133,0.749517,0.270411,...,0.026926,0.014663,0.013863,0.021861,0.015196,0.016796,0.012263,0.011730,0.020261,0.011197
std,0.079989,0.105860,0.078414,0.115885,0.102592,0.078702,0.090017,0.162731,0.071702,0.096128,...,0.161889,0.120215,0.116938,0.146249,0.122348,0.128522,0.110074,0.107683,0.140911,0.105236
min,0.000000,0.282128,0.000000,0.000000,0.002630,0.137873,0.006130,0.000000,0.275590,0.003040,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.033300,0.517811,0.000000,0.000000,0.138118,0.625627,0.207374,0.378062,0.707339,0.194357,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.066700,0.585989,0.050000,0.000000,0.190926,0.674037,0.277845,0.499942,0.738961,0.284316,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.100000,0.668395,0.100000,0.000000,0.261726,0.740663,0.335816,0.569962,0.788177,0.344626,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,0.964381,0.950000,1.000000,1.000000,0.994735,0.790831,0.989870,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
y_tr.head()

0    1
1    1
2    1
3    1
4    0
Name: Activity, dtype: int64

Exercise
---------------------

Using the starter code below try to improve the solution

1. What kind of models you can use?
2. Try changing model parameters to get the best cross validation error.
3. Use pipeline to transform features before modeling:
   - use some feature selection mechanism
   - use dimension reduction method (pca, svd, etc)
   
Tip: It is ok to loop over models and datasets like this.

```python
for data in [pipeline_1, pipeline_2, pipeline_3]:
    for model in [model_1, model_2, model_3]:
        # do stuff
```

### Logistic Regression

In [96]:
# Template model, for grid search
model_logreg=sk.linear_model.LogisticRegression(
    penalty='l2', 
                   dual=False, 
                   tol=0.0001, 
                   C=1.0, 
                   fit_intercept=True, 
                   intercept_scaling=1, 
                   class_weight=None, 
                   random_state=None, 
                   solver='liblinear', 
                   max_iter=100, 
                   multi_class='ovr', 
                   verbose=0, 
                   warm_start=False, n_jobs=1)

In [93]:
# Search over this template for best params
%%time
C_range = [0.01,0.1,0.5,1,1.5]
tol_range = [0.0001,0.001,0.01]
clf = GridSearchCV(model_logreg, 
                   param_grid={
                       'C':C_range,
                       'tol':tol_range})
clf.fit(X_tr,y_tr)
print(clf.best_params_)

{'C': 0.1, 'tol': 0.0001}
CPU times: user 38.6 s, sys: 3.66 s, total: 42.3 s
Wall time: 15.3 s


In [97]:
%%time
final_model = sk.linear_model.LogisticRegression(
                   tol=0.0001, 
                   C=.1,
                )

oof_predictions = cross_val_predict(clf, X_tr, y_tr, method="predict_proba")

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")


Your error is 0.5233482398905319
You can still improve :)
CPU times: user 1min 36s, sys: 8.3 s, total: 1min 44s
Wall time: 32.7 s


Make prediction on test data

In [47]:
%%script false
# Make the prediction
# THIS WON'T WORK SINCE
y_pred = model.predict(X_te)
score = y_te == y_pred
score = model.score(X_te, y_te)
sum(score)

### Stochastic Gradient Descent Classifer

In [126]:
# Template model, for grid search
model_sgd = SGDClassifier(
    loss='modified_huber', 
    penalty='l2', 
    alpha=0.0001,
    l1_ratio=0.15, 
    fit_intercept=True, 
    max_iter=None, 
    tol=None, 
    shuffle=True, 
    verbose=0, 
    epsilon=0.1, 
    n_jobs=1, 
    random_state=None, 
    learning_rate='optimal', 
    eta0=0.0, 
    power_t=0.5, 
    class_weight=None, 
    warm_start=False, 
    average=False, 
    n_iter=None)

In [127]:
alpha_range = np.logspace(0.00001,10,11)
alpha_range

array([1.00002303e+00, 1.00002072e+01, 1.00001842e+02, 1.00001612e+03,
       1.00001382e+04, 1.00001151e+05, 1.00000921e+06, 1.00000691e+07,
       1.00000461e+08, 1.00000230e+09, 1.00000000e+10])

In [128]:
%%script false
# Search over this template for best params%%time
#tol_range = [0.0001,0.001,0.01]
clf = GridSearchCV(model_sgd, 
                   param_grid={
                       'alpha':alpha_range,
                       })
clf.fit(X_tr,y_tr)
print(clf.best_params_)

In [133]:
#%%script false
#%%time
final_model = sk.linear_model.LogisticRegression(
                   tol=0.0001, 
                   C=.1,
                )

#oof_predictions = cross_val_predict(final_model, X_tr, y_tr, method="predict_proba")
oof_predictions = cross_val_predict(clf, X_tr, y_tr)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

Your error is 7.071740472159056
You can still improve :)


### Random Forest Classifier

In [122]:
%%script false
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression, BayesianRidge, SGDClassifier
from sklearn.pipeline import make_pipeline, make_union
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier


In [125]:
%%timeit
clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, n_estimators=100), 
                   param_grid={'max_depth':[5,10,15]})

oof_predictions = cross_val_predict(clf, X_tr, y_tr, method="predict_proba")

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

Your error is 0.4701962668251781
Your error is 0.46964817480586113
Your error is 0.47679303774244375
Your error is 0.47665307084158576
Your error is 0.4691921519395264
Your error is 0.4761401275916646
Your error is 0.46822251891078803
Your error is 0.4677563854744935
11.2 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Examples of classifiers 

<div class='spoiler'>

clf1 = RandomForestClassifier(n_estimators=10,n_jobs=-1)
clf2 = make_pipeline(PCA(), LogisticRegression())

clf3 = make_pipeline(
   make_union(
       RandomTreesEmbedding(n_estimators=10), 
       LazyTransformer()
   ), 
   LogisticRegression()
)

for clf in [clf1,clf2,clf3]:
    clf.fit(x,y)
    
clf = make_pipeline(make_union(make_pipeline(RandomTreesEmbedding(n_estimators=20), StandardScaler(with_mean=False)), 
                               make_pipeline(StandardScaler(with_mean=False), VarianceThreshold(0.1))),
                    MLPClassifier((15,), alpha=15.0, verbose=True))

clf = BaggingClassifier(make_pipeline(
                        make_union(RandomTreesEmbedding(n_estimators=10), 
                                   LazyTransformer()),
                        StandardScaler(with_mean=False), 
                        VarianceThreshold(0.001),
                        MLPClassifier((25,), alpha=10.0, verbose=False)), 
                        max_samples=0.75,
                        max_features=0.75,
                        n_estimators=10)
</div>